# Suraksha Setu — Advanced Multi-Hazard Models (Optuna + SMOTE + Feature Importance)

**This notebook includes**:
- Data loading for Flood, Landslide, and Forest Fire cleaned CSVs
- Preprocessing and feature engineering
- Class imbalance handling with SMOTE
- Model training with hyperparameter tuning using Optuna
  - Flood: LSTM (Keras) with Optuna tuner-like loop
  - Landslide: XGBoost with Optuna tuning
  - Forest Fire: CNN-LSTM with Optuna tuning
- Model saving (`.h5`, `.joblib`, `.json`)
- Stacking ensemble (XGBoost meta-model)
- Evaluation: ROC, Precision-Recall, confusion matrices, calibration, and feature importance
- Notes: Align time/spatial indices before real-world stacking


In [ ]:
# Install required packages (uncomment if running in Colab)
# !pip install xgboost tensorflow scikit-learn pandas numpy matplotlib seaborn joblib imbalanced-learn optuna tensorflow-addons

import os, random
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
sns.set(style='whitegrid')
import joblib
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import xgboost as xgb
import optuna
import warnings
warnings.filterwarnings('ignore')
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
print('Libraries loaded')

In [ ]:
# Paths - update if needed
FLOOD_PATH = '/mnt/data/flood_cleaned.csv'
LANDSLIDE_PATH = '/mnt/data/Landslide_Factors_cleaned.csv'
FIRE_PATH = '/mnt/data/forestfires_cleaned.csv'
OUTPUT_DIR = '/mnt/data/suraksha_models_advanced'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('OUTPUT_DIR =', OUTPUT_DIR)

In [ ]:
# Utility functions
from sklearn.calibration import calibration_curve
def print_metrics(y_true, y_pred_prob, threshold=0.5):
    y_pred = (y_pred_prob >= threshold).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc = roc_auc_score(y_true, y_pred_prob)
    ap = average_precision_score(y_true, y_pred_prob)
    print(f'Accuracy: {acc:.3f} | Precision: {prec:.3f} | Recall: {rec:.3f} | F1: {f1:.3f} | AUC: {auc:.3f} | AP: {ap:.3f}')
    return {'accuracy':acc,'precision':prec,'recall':rec,'f1':f1,'auc':auc,'ap':ap}

def plot_roc_pr(y_true, y_scores, label='Model'):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    ap = average_precision_score(y_true, y_scores)
    auc = roc_auc_score(y_true, y_scores)
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(fpr, tpr, label=f'{label} (AUC={auc:.3f})')
    plt.plot([0,1],[0,1],'k--', alpha=0.5)
    plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC Curve'); plt.legend()
    plt.subplot(1,2,2)
    plt.plot(recall, precision, label=f'{label} (AP={ap:.3f})')
    plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('Precision-Recall'); plt.legend()
    plt.show()

def save_keras_model(model, path):
    model.save(path)
    print('Saved Keras model to', path)

def save_joblib(obj, path):
    joblib.dump(obj, path)
    print('Saved joblib object to', path)

def plot_calibration(y_true, y_prob, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=n_bins)
    plt.plot(prob_pred, prob_true, marker='o', label='Calibration')
    plt.plot([0,1],[0,1],'k--', alpha=0.5)
    plt.xlabel('Predicted probability'); plt.ylabel('True probability'); plt.title('Calibration curve')
    plt.show()

In [ ]:
# Load datasets
df_flood = pd.read_csv(FLOOD_PATH)
df_land = pd.read_csv(LANDSLIDE_PATH)
df_fire = pd.read_csv(FIRE_PATH)
print('Flood:', df_flood.shape, 'Landslide:', df_land.shape, 'Fire:', df_fire.shape)

In [ ]:
# Flood preprocessing: prepare sequences and apply SMOTE at sequence-level via aggregated features
df = df_flood.copy()
# detect time and target
time_col = next((c for c in df.columns if 'date' in c.lower() or 'time' in c.lower()), None)
target_col = next((c for c in df.columns if c.lower() in ['risk','risk_score','target','label','flood_risk']), None)
if time_col: df[time_col] = pd.to_datetime(df[time_col], errors='coerce'); df = df.sort_values(time_col)
num_feats = df.select_dtypes(include='number').columns.tolist()
if target_col in num_feats: num_feats.remove(target_col)
print('Using numeric features:', num_feats)
# fill missing and scale
df[num_feats] = df[num_feats].fillna(df[num_feats].median())
scaler_flood = MinMaxScaler()
X_num = scaler_flood.fit_transform(df[num_feats])
save_joblib(scaler_flood, os.path.join(OUTPUT_DIR,'flood_scaler.joblib'))

# create rolling-window aggregates for SMOTE (since SMOTE on sequences is complex)
WINDOW = 24
agg_features = []
for i in range(WINDOW, len(df)):
    window = X_num[i-WINDOW:i]
    agg = np.concatenate([window.mean(axis=0), window.std(axis=0), window.min(axis=0), window.max(axis=0)])
    agg_features.append(agg)
agg_features = np.array(agg_features)
y = df[target_col].fillna(0).values[WINDOW:] if target_col else (agg_features[:,0] > np.percentile(agg_features[:,0],90)).astype(int)

print('Aggregated features shape for SMOTE:', agg_features.shape, 'y shape:', y.shape)
sm = SMOTE(random_state=RANDOM_SEED)
X_res, y_res = sm.fit_resample(agg_features, y)
print('After SMOTE:', X_res.shape, y_res.sum(), 'positive samples')

# Convert back to sequences by sampling from resampled aggregates - we'll build sequences by matching nearest aggregate index (approximation)
# For simplicity for training LSTM, we'll use the original sequences (without SMOTE) but use class weights computed from y_res
from collections import Counter
class_weights = {0: (len(y_res)/(2*Counter(y_res)[0])), 1: (len(y_res)/(2*Counter(y_res)[1]))}
print('Class weights for flood LSTM:', class_weights)

# Build actual sequences (original)
SEQ_LEN = 24
def create_sequences(X, y, seq_len=SEQ_LEN):
    Xs, ys = [], []
    for i in range(len(X)-seq_len):
        Xs.append(X[i:(i+seq_len)])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_num, df[target_col].fillna(0).values if target_col else (X_num[:,0] > np.percentile(X_num[:,0],90)).astype(int), SEQ_LEN)
split_idx = int(0.8 * len(X_seq))
X_train_flood, X_test_flood = X_seq[:split_idx], X_seq[split_idx:]
y_train_flood, y_test_flood = y_seq[:split_idx], y_seq[split_idx:]
print('Flood sequences:', X_train_flood.shape, y_train_flood.sum(), 'positives in train')

In [ ]:
# Optuna objective for LSTM (search over units, dropout, lr)
import optuna, math, tempfile
def build_lstm(trial, input_shape):
    units1 = trial.suggest_categorical('units1', [32,64,128])
    units2 = trial.suggest_categorical('units2', [16,32,64])
    dropout = trial.suggest_float('dropout', 0.1, 0.4)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    inp = layers.Input(shape=input_shape)
    x = layers.Bidirectional(layers.LSTM(units1, return_sequences=True))(inp)
    x = layers.Dropout(dropout)(x)
    x = layers.Bidirectional(layers.LSTM(units2))(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    out = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inp, out)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['AUC'])
    return model

def objective_lstm(trial):
    tf.keras.backend.clear_session()
    model = build_lstm(trial, (X_train_flood.shape[1], X_train_flood.shape[2]))
    batch = trial.suggest_categorical('batch_size', [32,64,128])
    epochs = 30
    history = model.fit(X_train_flood, y_train_flood, validation_split=0.1, epochs=epochs, batch_size=batch, class_weight=class_weights, verbose=0, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])
    preds = model.predict(X_test_flood).ravel()
    auc = roc_auc_score(y_test_flood, preds)
    # save intermediate best model
    return auc

study_lstm = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
study_lstm.optimize(objective_lstm, n_trials=12)  # adjust n_trials as compute allows
print('Best LSTM trial:', study_lstm.best_params, 'AUC:', study_lstm.best_value)

# Build final model with best params and save
best_params = study_lstm.best_params
final_model = build_lstm(optuna.trial.FixedTrial(best_params), (X_train_flood.shape[1], X_train_flood.shape[2]))
final_model.fit(X_train_flood, y_train_flood, validation_split=0.1, epochs=50, batch_size=best_params.get('batch_size',64), class_weight=class_weights, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])
save_keras_model(final_model, os.path.join(OUTPUT_DIR, 'flood_lstm_opt.h5'))
y_pred_prob_lstm = final_model.predict(X_test_flood).ravel()
metrics_lstm = print_metrics(y_test_flood, y_pred_prob_lstm)
plot_roc_pr(y_test_flood, y_pred_prob_lstm, label='Flood_LSTM_Opt')

In [ ]:
# Landslide preprocessing with SMOTE and feature scaling
df = df_land.copy()
target_col = next((c for c in df.columns if c.lower() in ['target','label','landslide','landslide_risk','susceptible']), None)
if target_col is None:
    # create proxy target
    num_cols = df.select_dtypes(include='number').columns.tolist()
    target_col = 'target'
    df[target_col] = (df[num_cols[0]] > np.percentile(df[num_cols[0]],90)).astype(int)
print('Target column for landslide:', target_col)
# Drop columns with too many missing
df = df.loc[:, df.isnull().mean() < 0.4]
num_cols = df.select_dtypes(include='number').columns.tolist()
if target_col in num_cols: num_cols.remove(target_col)
cat_cols = df.select_dtypes(include='object').columns.tolist()
for c in num_cols:
    df[c].fillna(df[c].median(), inplace=True)
for c in cat_cols:
    df[c].fillna('NA', inplace=True); df[c] = LabelEncoder().fit_transform(df[c].astype(str))
X = df[num_cols]; y = df[target_col].astype(int)
scaler_land = StandardScaler()
X_scaled = scaler_land.fit_transform(X)
save_joblib(scaler_land, os.path.join(OUTPUT_DIR,'land_scaler.joblib'))
sm = SMOTE(random_state=RANDOM_SEED)
X_res, y_res = sm.fit_resample(X_scaled, y)
print('After SMOTE:', X_res.shape, 'Positives:', y_res.sum())
X_train_land, X_test_land, y_train_land, y_test_land = train_test_split(X_res, y_res, test_size=0.2, random_state=RANDOM_SEED, stratify=y_res)

In [ ]:
# Optuna objective for XGBoost
def objective_xgb(trial):
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }
    model = xgb.XGBClassifier(**param, use_label_encoder=False, eval_metric='auc', random_state=RANDOM_SEED)
    model.fit(X_train_land, y_train_land, eval_set=[(X_test_land, y_test_land)], early_stopping_rounds=20, verbose=False)
    preds = model.predict_proba(X_test_land)[:,1]
    return roc_auc_score(y_test_land, preds)

study_xgb = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
study_xgb.optimize(objective_xgb, n_trials=30)
print('Best XGB params:', study_xgb.best_params, 'AUC:', study_xgb.best_value)

# Train final XGB with best params
best_xgb = xgb.XGBClassifier(**study_xgb.best_params, use_label_encoder=False, eval_metric='auc', random_state=RANDOM_SEED)
best_xgb.fit(X_train_land, y_train_land)
save_joblib(best_xgb, os.path.join(OUTPUT_DIR,'landslide_xgb.joblib'))

# Feature importance
fi = pd.DataFrame({'feature': df[num_cols].columns, 'importance': best_xgb.feature_importances_}).sort_values('importance', ascending=False)
plt.figure(figsize=(8,6)); sns.barplot(data=fi.head(20), x='importance', y='feature'); plt.title('Feature importance - Landslide XGB'); plt.show()

y_pred_prob_xgb = best_xgb.predict_proba(X_test_land)[:,1]
metrics_xgb = print_metrics(y_test_land, y_pred_prob_xgb)
plot_roc_pr(y_test_land, y_pred_prob_xgb, label='Landslide_XGB')

In [ ]:
# Fire data preprocessing: sequences + SMOTE via aggregated features
df = df_fire.copy()
time_col = next((c for c in df.columns if 'date' in c.lower() or 'time' in c.lower()), None)
target_col = next((c for c in df.columns if c.lower() in ['fire_risk','risk','hotspot','target','label']), None)
if time_col: df[time_col] = pd.to_datetime(df[time_col], errors='coerce'); df = df.sort_values(time_col)
num_feats = df.select_dtypes('number').columns.tolist()
if target_col in num_feats: num_feats.remove(target_col)
df[num_feats] = df[num_feats].fillna(df[num_feats].median())
scaler_fire = MinMaxScaler(); X_num = scaler_fire.fit_transform(df[num_feats]); save_joblib(scaler_fire, os.path.join(OUTPUT_DIR,'fire_scaler.joblib'))

# aggregated features for SMOTE
WINDOW = 12
agg = []
for i in range(WINDOW, len(df)):
    w = X_num[i-WINDOW:i]
    agg.append(np.concatenate([w.mean(axis=0), w.std(axis=0)]))
agg = np.array(agg)
y_agg = df[target_col].fillna(0).values[WINDOW:] if target_col else (agg[:,0] > np.percentile(agg[:,0],90)).astype(int)
sm = SMOTE(random_state=RANDOM_SEED); X_res_f, y_res_f = sm.fit_resample(agg, y_agg)
print('After SMOTE (fire):', X_res_f.shape, 'positives:', y_res_f.sum())

# Build sequences for CNN-LSTM (use original sequences for training, class weights from SMOTE)
SEQ_LEN = 12
def create_seq(X,y,seq_len=SEQ_LEN):
    Xs, ys = [], []
    for i in range(len(X)-seq_len):
        Xs.append(X[i:(i+seq_len)])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)
X_seq_f, y_seq_f = create_seq(X_num, df[target_col].fillna(0).values if target_col else (X_num[:,0] > np.percentile(X_num[:,0],90)).astype(int), SEQ_LEN)
split_idx = int(0.8 * len(X_seq_f))
X_train_fire, X_test_fire = X_seq_f[:split_idx], X_seq_f[split_idx:]
y_train_fire, y_test_fire = y_seq_f[:split_idx], y_seq_f[split_idx:]
from collections import Counter
cw_fire = {0: (len(y_res_f)/(2*Counter(y_res_f)[0])), 1: (len(y_res_f)/(2*Counter(y_res_f)[1]))}
print('Fire sequences:', X_train_fire.shape, 'class weights:', cw_fire)

In [ ]:
# Optuna for CNN-LSTM fire model
def build_fire_model(trial, input_shape):
    conv_filters = trial.suggest_categorical('conv_filters', [16,32,64])
    lstm_units = trial.suggest_categorical('lstm_units', [32,64])
    dropout = trial.suggest_float('dropout', 0.1, 0.4)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    inp = layers.Input(shape=input_shape)
    x = layers.TimeDistributed(layers.Conv1D(filters=conv_filters, kernel_size=3, activation='relu'))(inp)
    x = layers.TimeDistributed(layers.MaxPool1D(2))(x)
    x = layers.TimeDistributed(layers.Flatten())(x)
    x = layers.LSTM(lstm_units)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    out = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inp, out)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['AUC'])
    return model

def objective_fire(trial):
    tf.keras.backend.clear_session()
    model = build_fire_model(trial, (X_train_fire.shape[1], X_train_fire.shape[2]))
    batch = trial.suggest_categorical('batch_size', [32,64])
    history = model.fit(X_train_fire, y_train_fire, validation_split=0.1, epochs=30, batch_size=batch, class_weight=cw_fire, verbose=0, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])
    preds = model.predict(X_test_fire).ravel()
    return roc_auc_score(y_test_fire, preds)

study_fire = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
study_fire.optimize(objective_fire, n_trials=12)
print('Best fire params:', study_fire.best_params, 'AUC:', study_fire.best_value)

# Train final fire model with best params
best_fire_params = study_fire.best_params
final_fire = build_fire_model(optuna.trial.FixedTrial(best_fire_params), (X_train_fire.shape[1], X_train_fire.shape[2]))
final_fire.fit(X_train_fire, y_train_fire, validation_split=0.1, epochs=50, batch_size=best_fire_params.get('batch_size',32), class_weight=cw_fire, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])
save_keras_model(final_fire, os.path.join(OUTPUT_DIR,'fire_cnn_lstm_opt.h5'))
y_pred_prob_fire = final_fire.predict(X_test_fire).ravel()
metrics_fire = print_metrics(y_test_fire, y_pred_prob_fire)
plot_roc_pr(y_test_fire, y_pred_prob_fire, label='Fire_CNN_LSTM_Opt')

In [ ]:
# Stacking ensemble - ALIGN DATA BEFORE USING IN PRODUCTION!
# Here we create a demo meta-dataset by taking minimal overlapping samples.
len_f = len(y_test_flood) if 'y_test_flood' in globals() else 0
len_l = len(y_test_land) if 'y_test_land' in globals() else 0
len_fire = len(y_test_fire) if 'y_test_fire' in globals() else 0
min_len = min(len_f, len_l, len_fire) if min([len_f,len_l,len_fire])>0 else None
print('Test lengths (flood, land, fire):', len_f, len_l, len_fire, 'min_len:', min_len)

if min_len:
    meta_X = pd.DataFrame({
        'flood_prob': y_pred_prob_lstm[:min_len],
        'landslide_prob': y_pred_prob_xgb[:min_len],
        'fire_prob': y_pred_prob_fire[:min_len]
    })
    meta_y = y_test_flood[:min_len]  # proxy; align properly in real use
    meta_model = xgb.XGBClassifier(n_estimators=200, max_depth=3, random_state=RANDOM_SEED, use_label_encoder=False, eval_metric='logloss')
    meta_model.fit(meta_X, meta_y)
    save_joblib(meta_model, os.path.join(OUTPUT_DIR,'meta_xgb.joblib'))
    meta_prob = meta_model.predict_proba(meta_X)[:,1]
    metrics_meta = print_metrics(meta_y, meta_prob)
    plot_roc_pr(meta_y, meta_prob, label='Meta_Ensemble')

In [ ]:
# Comparison of model performance and visualizations
metrics_dict = {}
if 'metrics_lstm' in globals(): metrics_dict['Flood_LSTM'] = metrics_lstm
if 'metrics_xgb' in globals(): metrics_dict['Landslide_XGB'] = metrics_xgb
if 'metrics_fire' in globals(): metrics_dict['Fire_CNN_LSTM'] = metrics_fire
if 'metrics_meta' in globals(): metrics_dict['Meta_Ensemble'] = metrics_meta

df_metrics = pd.DataFrame(metrics_dict).T
display(df_metrics)

# Plot AUC and AP
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); df_metrics['auc'].plot(kind='bar', color='teal'); plt.title('AUC by Model'); plt.ylim(0,1)
plt.subplot(1,2,2); df_metrics['ap'].plot(kind='bar', color='orange'); plt.title('Average Precision (AP)'); plt.ylim(0,1)
plt.show()

# Calibration plots
for name, m in metrics_dict.items():
    probs = globals().get('y_pred_prob_'+ name.split('_')[0].lower(), None)
    true = globals().get('y_test_'+ name.split('_')[0].lower(), None)
    if probs is not None and true is not None:
        print('Calibration for', name)
        plot_calibration(true, probs)

In [ ]:
# Save scalers and list files
print('Saved models in', OUTPUT_DIR)
print('\nFiles:'); print('\n'.join(os.listdir(OUTPUT_DIR)))

print('\nNotes:')
print('- This notebook performs Optuna hyperparameter search; adjust n_trials for better tuning (compute/time tradeoff).')
print('- Before building a production meta-model, align samples by spatial/time keys (e.g., grid cell + timestamp).')
print('- Consider model calibration, ensembling weights, and uncertainty estimation for high-stakes alerts.')